# Proof-of-concept queries for Project Hetmech

Based on a highly customized _CbGiG_ metapath (_Compound-binds-Gene-interacts-Gene_).

In [1]:
from time import perf_counter

import pandas
from neo4j.v1 import GraphDatabase

## Connect to the [Hetionet v1.0 Neo4j instance](https://neo4j.het.io)

In [2]:
driver = GraphDatabase.driver("bolt://neo4j.het.io")
assert driver.encrypted

In [3]:
def run_query(query, parameters={}):
    """Execute a Cypher query and return results as a pandas.DataFrame"""
    start_time = perf_counter()
    with driver.session() as session:
        result = session.run(query, parameters)
        result_df = pandas.DataFrame((x.values() for x in result), columns=result.keys())
    runtime = perf_counter() - start_time
    return result_df, runtime

## Identify candidate treatments

The following query was prototyped on [GitHub Issues](https://github.com/greenelab/hetmech/issues/6#issuecomment-236287256). It looks for compound-disease pairs where:

+ the compound treats the disease
+ the disease has at least one GWAS-associated gene
+ the compound binds 2 or more genes
+ the compound binds a gene that interacts with a gene that is disease-associated
+ the compound does not bind a gene that is disease-associated (filter the obvious)

In [4]:
query = """
MATCH path = (compound:Compound)-[TREATS_CtD]-(disease:Disease)
WHERE
  exists((disease)-[:ASSOCIATES_DaG {unbiased: true}]-())
  AND size((compound)-[:BINDS_CbG]-()) > 1
  AND exists((compound)-[:BINDS_CbG]-()-[:INTERACTS_GiG]-()-[:ASSOCIATES_DaG {unbiased: true}]-(disease))
  AND NOT exists((compound)-[:BINDS_CbG]-()-[:ASSOCIATES_DaG {unbiased: true}]-(disease))
RETURN compound.name AS compound, disease.name AS disease
ORDER BY compound, disease
"""
treatment_df, query_time = run_query(query)
print('Neo4j query took {:.2f} seconds'.format(query_time))
treatment_df.tail(3)

Neo4j query took 10.70 seconds


,compound,disease
231,Vinorelbine,hematologic cancer
232,Vitamin C,prostate cancer
233,Vorinostat,hematologic cancer


## Infer the mechanistic targets for each treatment

For each treatment (compound-disease pair), we evaluate each target (gene that the compound binds) for whether it's the therapeutic mechanism. We base our inferrence of therapeutic mechanism by looking for targets which interact with GWAS-associated genes of the treated disease. We rank targets based on a precision assumption: the target with the greatest percentage of associations in its set of interacting genes (`precision`) is ranked highest.

Here are some definitions:

+ positives: the set of GWAS-associated genes for the disease
+ `TP + FP`: the set of interacting genes for the target that is bound by the compound
+ `TP` (true positives): the set of GWAS-associated genes that interact with the target 
+ `FP` (false positives): the set of GWAS-unassociated genes that interact with the target
+ `precision`: the percent of genes interacting with the target that are disease-associated

In [5]:
query = """
MATCH path = (compound:Compound)-[:BINDS_CbG]-(gene_1)-[:INTERACTS_GiG]-(gene_2)
  WHERE compound.name = { compound }
OPTIONAL MATCH (gene_2)-[association:ASSOCIATES_DaG {unbiased: true}]-(disease:Disease)
  WHERE disease.name = { disease }
WITH
  gene_1, gene_2, association
  ORDER BY gene_2.name
WITH
  gene_1,
  extract(x in collect(gene_2) | x.name) AS interactors,
  extract(x in collect(association) | endNode(x).name) AS associated_interactors
WITH
  gene_1,
  interactors, associated_interactors,
  size(associated_interactors) AS TP,
  size(interactors) AS `TP + FP`
RETURN
  { compound } AS compound,
  { disease } AS disease,
  gene_1.name AS target,
  associated_interactors,
  interactors,
  TP, `TP + FP`,
  toFloat(TP) / `TP + FP` AS precision
ORDER BY precision DESC, `TP + FP`, target
"""

dfs = list()
query_time = 0
for i, parameters in treatment_df.iterrows():
    parameters = dict(parameters)
    df, runtime = run_query(query, parameters)
    dfs.append(df)
    query_time += runtime

print('Neo4j queries took {:.2f} seconds'.format(query_time))

mechanism_df = pandas.concat(dfs)
for column in 'interactors', 'associated_interactors':
    mechanism_df[column] = mechanism_df[column].str.join(', ')

Neo4j queries took 9.41 seconds


In [6]:
# Number of treatment-target pairs
len(mechanism_df)

3505

In [7]:
mechanism_df.head()

,compound,disease,target,associated_interactors,interactors,TP,TP + FP,precision
0,Abiraterone,prostate cancer,ALB,KLK3,"ACADVL, ADRA1B, AGA, AHSG, AMBP, AMPD3, AP1M1,...",1,165,0.006061
1,Abiraterone,prostate cancer,ABCC1,,"AZGP1, NPM1, TRAF3",0,3,0.000000
2,Abiraterone,prostate cancer,SULT2A1,,"PRDX4, STAT3, STK3, SULT1B1, TMX1, UBD",0,6,0.000000
3,Abiraterone,prostate cancer,CYP3A4,,"AMFR, CYB5A, GK, PGRMC1, POR, STUB1, UGT2B7",0,7,0.000000
4,Abiraterone,prostate cancer,ORM1,,"ALB, BLVRB, CCR5, GLUD1, GSDMB, PIK3R3, SERPIN...",0,8,0.000000


In [8]:
mechanism_df.to_csv('CbGiG-candidates.tsv', index=False, sep='\t', float_format='%.3g')